# LangChain to prompt programmatically in Python

1. Create an account on [OpenAI Platform](https://platform.openai.com/)
2. Create a project
3. Create an API Key in the project (Settings > Project > API Keys)
4. Paste it into the `.env` file under the variable `OPENAI_API_KEY`

## Load model

In [1]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="chatgpt-4o-latest")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff3ad8a2d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff3ada8710>, root_client=<openai.OpenAI object at 0xffff3b729c70>, root_async_client=<openai.AsyncOpenAI object at 0xffff3ad8a330>, model_name='chatgpt-4o-latest', model_kwargs={}, openai_api_key=SecretStr('**********'))

## Prompting seralization

### Load markdown template

In [2]:
with open('../prompts/ES.md', 'r') as file:
    template = file.read()
    
template

'La siguiente tabla muestra los retornos del activo financiero con ticker {TICKER}, calculados como la variación porcentual entre el precio de cierre de cada fecha y el de la fecha inmediatamente anterior registrada en la tabla. La frecuencia aplicada al cálculo ha sido definida como {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nTABLA:\n\n{TABLE}\n\nExplica los motivos detrás de los valores más extremos (tanto positivos como negativos), utilizando noticias relevantes que puedan justificar dichos movimientos. Para cada valor, proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.'

### Create prompt object

In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    template_format='f-string',
)

prompt

PromptTemplate(input_variables=['FREQUENCY', 'TABLE', 'TICKER'], input_types={}, partial_variables={}, template='La siguiente tabla muestra los retornos del activo financiero con ticker {TICKER}, calculados como la variación porcentual entre el precio de cierre de cada fecha y el de la fecha inmediatamente anterior registrada en la tabla. La frecuencia aplicada al cálculo ha sido definida como {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nTABLA:\n\n{TABLE}\n\nExplica los motivos detrás de los valores más extremos (tanto positivos como negativos), utilizando noticias relevantes que puedan justificar dichos movimientos. Para cada valor, proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')

### Save prompt to JSON

In [4]:
path = '../prompts/ES.json'
prompt.save(path)

## Chaining with LLMs

### Define chain

In [5]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()

chain

PromptTemplate(input_variables=['TABLA', 'TICKER'], input_types={}, partial_variables={}, template='La siguiente tabla muestra el retorno del activo financiero con ticker {TICKER}, calculado como la variación porcentual entre el precio de cierre de cada fecha y el de la fecha inmediatamente anterior registrada en la tabla. Esta comparación se realiza respetando la frecuencia temporal constante observada en la serie (por ejemplo, diaria, semanal o mensual).\n\nTABLA:\n\n{TABLA}\n\nAnaliza los valores extremos, tanto positivos como negativos, y proporciona una explicación fundamentada para cada uno. Incluye, en la medida de lo posible, noticias relacionadas que puedan justificar dichas variaciones, junto con su fecha de publicación y un enlace a la fuente.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff6528df40>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff652ac440>, root_client=<openai.Ope

### Preprocess input data

In [1]:
ticker = 'AAPL'
freq = 'D'

In [7]:
import utils

df = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq=freq,
    n=5
)

df

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Ticker,AAPL
Date,
2024-01-03,-0.748750
2024-01-04,-1.270022
...,...
2024-12-27,-1.324213
2024-12-30,-1.326343


### Invoke chain with input data

In [8]:
output = chain.invoke({'TABLA': df, 'TICKER': ticker, 'FREQUENCY': freq})
print(output)

Para analizar los valores extremos (máximos positivos y mínimos negativos) del retorno diario del activo financiero con ticker AAPL (Apple Inc.), primero identificaremos las fechas y valores donde se produjeron los movimientos más significativos en el precio. Luego, contextualizaremos dichos movimientos con noticias relevantes que puedan explicar estas variaciones.

Nota: Dado que solo se proporcionan los extremos de la tabla (primeras y últimas fechas), y se indica que hay 250 filas (lo que sugiere una frecuencia diaria para cada día hábil del año 2024), se asumirá que los datos se refieren a todo el año bursátil 2024. Para esta respuesta, se utilizarán datos reales disponibles hasta la fecha de corte de conocimiento (junio de 2024) y se incluirán justificaciones plausibles para el resto del año.

I. Identificación de valores extremos

Para encontrar los valores más extremos (positivos y negativos), simulamos que realizamos la búsqueda sobre la columna de retornos:

- Retorno positivo

## Export output to file

In [10]:
with open(f'reports/{ticker}.md', 'w') as file:
    file.write(output)